In [2]:
import json
import os
import csv
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
# Input and Output directories
input_dir = "/Users/JB/Rotation_bkslab/250203_alphafold3/JSON_files"
output_dir = "/Users/JB/Rotation_bkslab/250203_alphafold3/JSON_pocket"
csv_file = "/Users/JB/Rotation_bkslab/250203_alphafold3/pocket_plddt.csv"
os.makedirs(output_dir, exist_ok=True)

total_plddt = []

for json_file in glob.glob(os.path.join(input_dir, "*.json")):
    with open(json_file, "r") as file:
        data = json.load(file)
    atom_chain_ids = data.get("atom_chain_ids", [])
    atom_plddts = data.get("atom_plddts", [])
    ref_prefix = os.path.splitext((os.path.basename(json_file)))[0]

    filtered_chain_ids = []
    filtered_plddts = []
    
    for i, (chain_id, plddt) in enumerate(zip(atom_chain_ids, atom_plddts)):
        if chain_id == "A" and (147 <= i <= 167 or 347 <= i <= 357 or 367 <= i <= 371 or 914 <= i <= 928 or 943 <= i <= 951 or 1133 <= i <= 1163 or 1181 <= i <= 1188):
            filtered_chain_ids.append(chain_id)
            filtered_plddts.append(plddt)
    
    average_plddt = sum(filtered_plddts) / len(filtered_plddts) if filtered_plddts else 0
    
    data["atom_chain_ids"] = filtered_chain_ids
    data["atom_plddts"] = filtered_plddts
    data["average_plddt"] = average_plddt
    total_plddt.append([ref_prefix, average_plddt])
    
    # Generate the output file path
    output_file_path = os.path.join(output_dir, os.path.basename(json_file))

    # Save the modified JSON
    with open(output_file_path, "w") as output_file:
        json.dump(data, output_file, indent=4)

# Save results to CSV
with open(csv_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Protein", "Pocket-pLDDT"])
    writer.writerows(total_plddt)

print(f"[INFO] Pocket-pLDDT results saved to {csv_file}")

[INFO] Pocket-pLDDT results saved to /Users/JB/Rotation_bkslab/250203_alphafold3/pocket_plddt.csv


In [49]:
csv_file = "/Users/JB/Rotation_bkslab/250203_alphafold3/ligand_plddt.csv"
input_dir = "/Users/JB/Rotation_bkslab/250203_alphafold3/JSON_ligands"

for json_file in glob.glob(os.path.join(input_dir, "*.json")):
    with open(json_file, "r") as file:
        data = json.load(file)